In [1]:
import numpy as np
from scipy.signal import convolve
from scipy.ndimage import shift as ndshift
from pprint import pprint

In [2]:
np.set_printoptions(linewidth=250)
np.set_printoptions(precision=3)

# Standard fshift function

In [3]:
def fshift(arr, lagx, lagy):
    """
    Shifts a 2D array (casted to float) with fractional shifts using SciPy.
    
    Note: SciPy's shift convention is (y, x), but our images are already in the right shape
    """
    return ndshift(arr, (lagx, lagy), 'float', order=1, prefilter=True, mode='grid-constant', cval=0.0)

# Standard erosion function

In [4]:
def ferosion( arr,step,cut):
    """
    2D matrix erosion for simulating finite thickness effect in shadow projections.
    It takes a mask array and "thins" the mask elements across the columns' direction.
    """
    # number of bins to cut
    ncuts = int(cut / step)# + int(np.sign(cut))
    print('ncuts', ncuts)
    print('cut', cut)
    
    arr_mask = (arr > 0) & (fshift(arr, ncuts , 0 ) > 0)
    cutted = arr * arr_mask if ncuts else arr

    # array indexes to be fractionally reduced:
    #   - the bin with the decimal values is the one
    #     to the left or right wrt the cutted bins
    erosion_value =  abs(cut / step - ncuts)
    print('erosion value', erosion_value)
    
    cutted_mask = (
        np.array((cutted > 0), dtype=int) - np.array((fshift(cutted, int(np.sign(cut)), 0) > 0), dtype=int)
    )
    border = (cutted_mask > 0)
    return cutted * (1.0 - border * erosion_value)

# Dummy mask and camera properties

In [5]:
mask =np.array( [
    [0,0,0, 0,0,0,0,0, 1,1,1,1,1, 0,0,0,0,0, 0,0,0],
    [0,0,0, 0,0,0,0,0, 1,1,1,1,1, 0,0,0,0,0, 0,0,0],
    [0,0,0, 0,0,0,0,0, 1,1,1,1,1, 0,0,0,0,0, 0,0,0]
])

In [6]:
MTHICK = 30
focal = 60 + MTHICK
theta = -20.0
ELXDIM = 10.0
shiftX = -focal * np.tan(np.deg2rad(theta))/ELXDIM
print("Shift in pixels:", shiftX)

Shift in pixels: 3.275732108395821


<img src="new_mask_erosion_screenshot.png"></img>

# We need to calculate the "farest border", i.e. the border of the shifted mask projection farest from the center
## This is the right border for shift>0 and the left border for shift<0

In [7]:
nearest_border =1 - abs(shiftX - int(shiftX)) #this is not really needed, just for check
print("nearest border:", nearest_border) #this is not really needed, just for check

farest_border = abs(shiftX - int(shiftX)) #this is needed!
print("farest border:", farest_border) #this is needed!

nearest border: 0.7242678916041791
farest border: 0.27573210839582085


# Now we estimate the reduction factor due to the vignetting
## Note that this factor is calculated on the standard pixel grid (not the fshited one)

In [8]:
angle_x_rad = -np.arctan(shiftX*ELXDIM / focal)
red_factor_x = MTHICK * np.tan(angle_x_rad)
print("red_factor_x:", red_factor_x)

red_factor_x: -10.91910702798607


# Now we need to re-align the reduction factor to the closer "external" pixel edge

In [9]:
red_factor_x_corr = red_factor_x + np.sign(red_factor_x) * (1- farest_border)*ELXDIM
print("red_factor_x_corr:", red_factor_x_corr)

red_factor_x_corr: -18.161785944027862


# With these numbers we can now fshift the mask array (here is 1D for simplicity)

In [10]:
mask_shifted = fshift(mask, 0, shiftX)
print(mask_shifted)

[[0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.724 1.    1.    1.    1.    0.276 0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.724 1.    1.    1.    1.    0.276 0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.724 1.    1.    1.    1.    0.276 0.    0.    0.    0.   ]]


# And erode this shifet mask with the correct red_factor_x_corr

In [11]:
mask_vignetted = ferosion(mask_shifted.T, ELXDIM, red_factor_x_corr)
print()
print(mask_vignetted.T)

ncuts -1
cut -18.161785944027862
erosion value 0.8161785944027862

[[0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.724 1.    1.    1.    0.184 0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.724 1.    1.    1.    0.184 0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.724 1.    1.    1.    0.184 0.    0.    0.    0.    0.   ]]
